This file displays the code for building the baseline LSTM model and then using it to generate text for each celebrity. 

The steps are as follows:

1. Load the data for the celebrity in question
2. Convert the text into pairs of sequences and output characters that will serve as the input and output of the model respectively
3. Build the LSTM model (no complexities).
4. Train the model with the processed data.
5. Use the model to generate text.
6. Export the generated text with pickle. 
7. Repeat steps 1-6 for all celebrities

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print('File not in drive')

File not in drive


In [ ]:
# import libraries
import pickle
import random
import time
import os
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, LambdaCallback
import sys

## Import Data

In [ ]:
# load preprocessed dataset
pickle_in = open('df.pickle', 'rb')
df = pickle.load(pickle_in)
df.head()


,Username,User handle,Date of posting,Text,Retweet count,Like count,Occupation,Text (EPA),Text (Model),Word Count
0,Alicia Keys,@aliciakeys,Fri Feb 12 03:16:07 +0000 2021,The maestro! The musical magician! The one and...,170,1973,Artist,maestro musical magician believe knew played s...,the maestro! the musical magician! the one and...,40
1,Alicia Keys,@aliciakeys,Wed Feb 10 21:31:09 +0000 2021,"Your glow is about to be on 100,000!!! ✨As we ...",101,1171,Artist,"glow 100,000 lead reminder love wait luminous ...","your glow is about to be on 100,000!!! as we l...",54
3,Alicia Keys,@aliciakeys,Wed Feb 10 01:32:56 +0000 2021,Woke up in such a good vibe.⁣ Gen was funky &a...,267,3659,Artist,woke good vibe.⁣ funky fussy caught energy thi...,woke up in such a good vibe. gen was funky &am...,52
4,Alicia Keys,@aliciakeys,Mon Feb 08 01:41:28 +0000 2021,One of my favorite small businesses is @unionl...,144,816,Artist,favorite small businesses unionlosangeles repl...,one of my favorite small businesses is@unionlo...,33
5,Alicia Keys,@aliciakeys,Mon Feb 08 01:22:00 +0000 2021,Dreaming of performing live \nfor you!!! ✨✨✨✨\...,275,5028,Artist,dreaming performing live city come meet tonight,dreaming of performing live for you!!! what ci...,21


In [ ]:
# convert list of texts to one corpus
def corpus(df):
  corpus = ""
  for text in df['Text (Model)']:
    corpus = corpus + str(text)
  return corpus


In [ ]:
# create a list of names
names = sorted(list(df['Username'].value_counts().index))
print(names)

['Alicia Keys', 'Anthony Joshua', 'Barack Obama', 'Bill Gates', "Conan O'Brien", 'Donald Trump', 'Dwayne Johnson', 'Elizabeth Warren', 'Ellen DeGeneres', 'Elon Musk', 'Emma Watson', 'Gordon Ramsay', 'Harry Styles.', 'Jeff Weiner', 'Joe Biden', 'John Cena', 'Kevin Durant', 'Kevin Hart', 'Kylie Jenner', 'Lady Gaga', 'LeBron James', 'Louis Tomlinson', 'Mariah Carey', 'Neil Patrick Harris', 'Oprah Winfrey', 'Pope Francis', 'Ronda Rousey', 'Tim Cook', 'Wiz Khalifa', 'daniel tosh', 'jimmy fallon']


In [ ]:
# create a corpus for each celebrity
corpuses = []
for name in names:
  df_name = df[df['Username'] == name]
  corpus_name = corpus(df_name)
  corpuses.append(corpus_name)

# store names and corpuses in a new data frame
df_corpus = pd.DataFrame()
df_corpus['Name'] = names
df_corpus['Corpus'] = corpuses

In [ ]:
# create a list of all characters
 
 
 chars = sorted(list(set(''.join(tweets))))
 print('Number of characters: ',len(chars))

# create a dictionary assigning each character to a number
 char_number= dict((c,i) for i,c in enumerate(chars))

 # create a dictionary assigning each number to a character
 number_char = dict((i,c) for i,c in enumerate(chars))

Number of characters:  58


In [ ]:
# constants
LENGTH = 50 # sequence length
STEP = 1

# input
sentences = []

# output
next_char = []

# create sequences and their corresponding output 

for x in tweets:
  for i in range(0, len(x)- LENGTH, STEP):
    sentences.append(x[i:i+LENGTH])
    next_char.append(x[i+LENGTH])


print('Number of sequences: ',len(sentences))

Number of sequences:  46582


## Vectorization

In [ ]:
# create input and output arrays
x = np.zeros((len(sentences), LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

# assign 1 for characters that are present in the char_number dictionary
for i, sentence in enumerate(sentences):
  for j, character in enumerate(sentence):
    x[i, j, char_number[character]] = 1
  y[i, char_number[next_char[i]]] = 1

In [ ]:
# show x and y dimensions

print('Dimensions of x: ',x.shape)
print('Dimensions of y: ',y.shape)

Dimensions of x:  (46582, 50, 58)
Dimensions of y:  (46582, 58)


## Model

## Baseline Model

In [ ]:
# build baseline model
baseline_model = Sequential()
baseline_model.add(LSTM(256, input_shape=(LENGTH, len(chars))))
baseline_model.add(Dropout(0.25))
baseline_model.add(Dense(len(chars), activation='softmax'))

baseline_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [ ]:
# create a sample text with the model
# temperature variable --> creativity, the higher, the more creative
def create_sample(prediction, temperature = 1):
  prediction = np.asarray(prediction).astype('float64')
  prediction = np.log(prediction)/temperature
  exp_preds = np.exp(prediction)
  prediction = exp_preds/np.sum(exp_preds)
  probability = np.random.multinomial(1,prediction, 1)
  return np.argmax(probability)

In [ ]:
# print text with generated sentences

def print_text(epoch, _):
  print()
  print('Generating Text after Epoch ',epoch)

  tweet = np.random.choice(tweets)
  start_index = 0
  for val in [0.5, 0.75 ,1.0, 1.25]:
    print('Value: ', val)

    generated = ''
    sentence = tweet[start_index: start_index+LENGTH]
    generated += sentence

    print('Generated sentence: "',sentence+'"')
    sys.stdout.write(generated)

    for i in range(110):
      x_pred = np.zeros((1, LENGTH, len(chars)))
      for j, character in enumerate(sentence):
        x_pred[0, j, char_number[character]] = 1

      prediction = baseline_model.predict(x_pred, verbose=0)[0]
      next_index = create_sample(prediction, val)
      next_char = number_char[next_index]

      generated += next_char
      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()
    print()


In [ ]:
# train model

EPOCHS = 20

print_callback = LambdaCallback(on_epoch_end=print_text)

baseline_model.fit(x, y, batch_size=128, epochs=EPOCHS, callbacks=[print_callback])

Epoch 1/20
364/364 [==============================] - 117s 318ms/step - loss: 3.2079

Generating Text after Epoch  0
Value:  0.5
Generated sentence: " news &amp; jokes for thursday 8/13/20. #fallontoni"
news &amp; jokes for thursday 8/13/20. #fallontonigh  and ona corsmint afe ong ingt !e yosane fon at aon one ar ine aou antinit an  fol on ond fani the yo wand 
Value:  0.75
Generated sentence: " news &amp; jokes for thursday 8/13/20. #fallontoni"
news &amp; jokes for thursday 8/13/20. #fallontonigw#fatlonsanshp wi ion ffolonthncone  wo lardonig thmrriin #sa ofaelonght wiy ahgad lcon. forione tfos and yor
Value:  1.0
Generated sentence: " news &amp; jokes for thursday 8/13/20. #fallontoni"
news &amp; jokes for thursday 8/13/20. #fallontonieut n toisthwos jakst es tyfrcileor,yonitk! wi.! #ulee tr! fon s y, asu@onal. and yu.iigpirgh7. remmw@sona .os
Value:  1.25
Generated sentence: " news &amp; jokes for thursday 8/13/20. #fallontoni"
news &amp; jokes for thursday 8/13/20. #fallontonit th

In [ ]:
# create text with a given sentence and diversity  value

def generate_text(sentence, diversity):
  sentence = sentence[0: LENGTH]
  print('sentence: ',sentence)
  print('diversity: ',diversity)

  generated = ''
  generated += sentence
  text_generated = ''
  sys.stdout.write(generated)

  for i in range(120):
    x_pred = np.zeros((1, LENGTH, len(chars)))
    for j, character in enumerate(sentence):
      x_pred[0, j, char_number[character]] = 1
    
    prediction = baseline_model.predict(x_pred, verbose=0)[0]
    next_index = create_sample(prediction, diversity)
    next_char = number_char[next_index]

    generated += next_char
    text_generated += next_char  
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()
  return text_generated


      


In [ ]:
generated_texts = []

# create sentences and add them to the generated_texts list
for sample in random.sample(list(tweets), 30):
  for diversity in [ 0.75]:
    generated_texts.append(generate_text(sample, diversity))
    print()

sentence:  the tonight show: at home edition (arnold @schwarz
diversity:  0.75
the tonight show: at home edition (arnold @schwarzzedeggad @therotstrass, and i storgturt and friends. thanks keroo soonat. ove you day  #fallontonight #thatpots2#fallon 

sentence:  fun show tonight with a very special guest! plus, 
diversity:  0.75
fun show tonight with a very special guest! plus, @righar @nackmunerlco @allersinnol. sup your funny by @alyofiecx &amp; me in the subway?!!! #gassafanout faglivis. what 

sentence:  every friday leading up to black friday @dopequeen
diversity:  0.75
every friday leading up to black friday @dopequeenphears #fallontonight #fallonmontent #fallonmonogit! #the askay #fallonisht #fallontonight thinks! #fallontonight #nacks

sentence:  were on tonight! like soon... #fallontonight #nbc 
diversity:  0.75
were on tonight! like soon... #fallontonight #nbc 11:35pm @ranyonmuncell. ith thank you more for way loved borks!!!! him manigite soorm co gave to and it in. hould a ge

## Save Generated Text as Pickle File

In [ ]:
# export generated text

text_pickle_out = open("fallon_text_generation_baseline.pickle", 'wb')
pickle.dump(generated_texts, text_pickle_out)
text_pickle_out.close()